In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

我们再来看看`Index`类型，它为`Series`和`DataFrame`对象提供了索引服务，有了索引我们就可以排序数据（`sort_index`方法）、对齐数据（在运算和合并数据时非常重要）并实现对数据的快速检索（索引运算）。由于`DataFrame`类型表示的是二维数据，所以它的行和列都有索引，分别是`index`和`columns`。`Index`类型的创建的比较简单，通常给出`data`、`dtype`和`name`三个参数即可，分别表示作为索引的数据、索引的数据类型和索引的名称。由于`Index`本身也是一维的数据，索引它的方法和属性跟`Series`非常类似，你可以尝试创建一个`Index`对象，然后尝试一下之前学过的属性和方法在`Index`类型上是否生效。接下来，我们主要看看`Index`的几种子类型。

### 范围索引

范围索引是由具有单调性的整数构成的索引，我们可以通过`RangeIndex`构造器来创建范围索引，也可以通过`RangeIndex`类的类方法`from_range`来创建范围索引，代码如下所示。

In [3]:
sales_data = np.random.randint(400, 1000, 12)
index = pd.RangeIndex(1, 13, name="月份")
ser = pd.Series(data=sales_data, index=index)
ser

月份
1     651
2     614
3     686
4     604
5     558
6     969
7     808
8     616
9     706
10    767
11    695
12    456
dtype: int64

### 分类索引

分类索引是由定类尺度构成的索引。如果我们需要通过索引将数据分组，然后再进行聚合操作，分类索引就可以派上用场。分类索引还有一个名为`reorder_categories`的方法，可以给索引指定一个顺序，分组聚合的结果会按照这个指定的顺序进行呈现，代码如下所示。

In [4]:
sales_data = [6, 6, 7, 6, 8, 6]
index = pd.CategoricalIndex(
    data=[
        "苹果",
        "香蕉",
        "苹果",
        "苹果",
        "桃子",
        "香蕉",
    ],  # data必须和上面的sales_data长度一致，data一般不会叫你手动敲，这里只是演示，一般可能是传递list或者Series（不能是dict，因为dict是键值对）
    categories=[
        "苹果",
        "香蕉",
        "桃子",
    ],
    ordered=True,  # 让categories可以比较大小。比如在这里，苹果<香蕉<桃子
)
ser = pd.Series(data=sales_data, index=index)
ser

苹果    6
香蕉    6
苹果    7
苹果    6
桃子    8
香蕉    6
dtype: int64

## 有了 ordered=True，就能比较 categories 了对吧，请问这个比较有什么意义吗？

### 比较的实际意义

排序控制：

- 分组聚合结果按你定义的顺序显示，而不是字母序
- 例如：客户等级（青铜 < 白银 < 黄金）、满意度（差 < 一般 < 好）

筛选操作：
  `ser[ser.index >= "香蕉"] # 筛选出"香蕉"级别以上的数据`

统计分析：

- 计算中位数、分位数等需要顺序的统计量
- 有序分类可以用于趋势分析

对于水果例子确实意义不大，但对于等级、评分、时期等天然有顺序的分类数据很有用。

## 如果没有 categories 那一行会怎么样？

如果没有此行，代码也能运行，因为 pandas 会自动从 data 中推断出所有唯一值作为 categories。但是，如果没有 categories，你可以试一下将 data 中的某一个分类改为一个其他的无意义的字符串，它照样显示出来了，如果有了 categories，规定只能是：苹果，香蕉，桃子这三类，那么那个无意义的字符串就会显示成 NaN


基于索引分组数据，然后使用`sum`进行求和。

In [5]:
ser.groupby(level=0, observed=True).sum()

苹果    19
香蕉    12
桃子     8
dtype: int64

## 1. 怎么体现"基于索引分组数据"？
`groupby(level=0)` 中的 `level=0` 指的是索引的第一层。在你的数据中，索引就是水果名称（苹果、香蕉、桃子），所以这里是按照索引中的水果类别进行分组。
## 2. level=0 有什么用？
`level=0` 表示按照索引的第0层（第一层）分组：
- 如果索引是单层的，level=0 就是整个索引
- 如果是多层索引（MultiIndex），level=0 是最外层，level=1 是第二层，以此类推
## 3. 为什么要设置 observed=True？
`observed=True` 的作用是只对实际存在的分类进行分组：
- `observed=False`（默认）：会包含所有定义的 categories，即使数据中没有出现
- `observed=True`：只包含数据中实际出现的分类

在你的例子中，虽然定义了三个 categories（苹果、香蕉、桃子），但如果某个分类在数据中不存在，设置 observed=True 可以避免在结果中显示空的分组，提高性能。不信你可以试一下，将CategoricalIndex的data中的某个“苹果”的地方改为一个无意义的字符串，因为有categories的存在，上面的groupby后的sum求和操作不会把这个无意义的字符串算进来，而是相应的，苹果的数量变少了（因为其中有一个变成了无意义的字符串了）

---


指定索引的顺序。

In [6]:
ser.index = index.reorder_categories(["香蕉", "桃子", "苹果"])
ser.groupby(level=0, observed=True).sum()

香蕉    12
桃子     8
苹果    19
dtype: int64

### 多级索引

Pandas 中的`MultiIndex`类型用来表示层次或多级索引。可以使用`MultiIndex`类的类方法`from_arrays`、`from_product`、`from_tuples`等来创建多级索引，我们给大家举几个例子。

In [7]:
tuples = [(1, "red"), (1, "blue"), (2, "red"), (2, "blue")]
index = pd.MultiIndex.from_tuples(tuples, names=["no", "color"])
index

MultiIndex([(1,  'red'),
            (1, 'blue'),
            (2,  'red'),
            (2, 'blue')],
           names=['no', 'color'])

In [8]:
arrays = [[1, 1, 2, 2], ["red", "blue", "red", "blue"]]
index = pd.MultiIndex.from_arrays(arrays, names=["no", "color"])
index

MultiIndex([(1,  'red'),
            (1, 'blue'),
            (2,  'red'),
            (2, 'blue')],
           names=['no', 'color'])

## 这个“多级索引”的多级体现在哪里？
多级索引的"多级"体现在索引具有层次结构：
- 第一级（level=0）: `no`列 - 值为 `[1, 1, 2, 2]`
- 第二级（level=1）: `color`列 - 值为 `["red", "blue", "red", "blue"]`

每个数据点的索引不是单一值，而是一个元组组合：`(1, "red")、(1, "blue")、(2, "red")、(2, "blue")`

这样可以用`两个维度来唯一标识数据`，类似于二维表格的行列坐标系统。在分组时可以按不同级别分组：
- `groupby(level=0)` → 按 `no` 分组
- `groupby(level=1)` → 按 `color` 分组

In [9]:
sales_data = np.random.randint(1, 100, 4)
ser = pd.Series(data=sales_data, index=index)
ser

no  color
1   red      47
    blue     73
2   red      14
    blue     55
dtype: int64

In [10]:
ser.groupby("no").sum()

no
1    120
2     69
dtype: int64

In [11]:
ser.groupby(level=1, observed=True).sum()  # level=1就是按照color分组

color
blue    128
red      61
dtype: int64

In [12]:
stu_ids = np.arange(1001, 1006)
print(stu_ids)
semisters = ["期中", "期末"]
index = pd.MultiIndex.from_product((stu_ids, semisters), names=["学号", "学期"])
print(index)
courses = ["语文", "数学", "英语"]
scores = np.random.randint(60, 101, (10, 3))
df = pd.DataFrame(data=scores, columns=courses, index=index)
df

[1001 1002 1003 1004 1005]
MultiIndex([(1001, '期中'),
            (1001, '期末'),
            (1002, '期中'),
            (1002, '期末'),
            (1003, '期中'),
            (1003, '期末'),
            (1004, '期中'),
            (1004, '期末'),
            (1005, '期中'),
            (1005, '期末')],
           names=['学号', '学期'])


语文   数学  英语
学号   学期              
1001 期中  100   81  87
     期末   79   62  88
1002 期中   90   77  73
     期末   93   90  71
1003 期中   85  100  71
     期末   96  100  69
1004 期中   80   64  65
     期末   89   69  72
1005 期中   65   73  91
     期末   74   88  73

## 📚 代码执行流程详细分析

### 🔍 整体目标
这段代码创建了一个**学生成绩表**，使用**多级索引**来表示不同学生在不同考试期间的成绩数据。

### 📋 逐步执行流程

#### 第1步：生成学号数组
```python
stu_ids = np.arange(1001, 1006)
print(stu_ids)
```
**作用**: 生成连续的学号数组  
**输出**: `[1001 1002 1003 1004 1005]`  
**说明**: 5个学生，学号从1001到1005

#### 第2步：定义考试学期
```python
semisters = ["期中", "期末"]
```
**作用**: 定义两个考试时期  
**输出**: `["期中", "期末"]`

#### 第3步：创建多级索引 🎯
```python
index = pd.MultiIndex.from_product((stu_ids, semisters), names=["学号", "学期"])
print(index)
```
**核心功能**: `from_product()` 创建**笛卡尔积**  
**输出**:
```
MultiIndex([(1001, '期中'),
            (1001, '期末'),
            (1002, '期中'),
            (1002, '期末'),
            (1003, '期中'),
            (1003, '期末'),
            (1004, '期中'),
            (1004, '期末'),
            (1005, '期中'),
            (1005, '期末')],
           names=['学号', '学期'])
```

#### 第4步：定义课程列
```python
courses = ["语文", "数学", "英语"]
```
**作用**: 设定DataFrame的列名

#### 第5步：生成随机成绩数据
```python
scores = np.random.randint(60, 101, (10, 3))
```
**作用**: 生成10行×3列的随机整数矩阵  
**范围**: 60-100分  
**维度**: (10, 3) → 10个索引组合 × 3门课程

#### 第6步：创建最终DataFrame
```python
df = pd.DataFrame(data=scores, columns=courses, index=index)
```
**结果**: 多级索引的学生成绩表

### 🔧 from_product() 详解

#### 什么是笛卡尔积？
`from_product()` 计算**笛卡尔积**，即两个集合中每个元素的所有可能组合：

```python
# 输入两个列表
stu_ids = [1001, 1002, 1003, 1004, 1005]
semisters = ["期中", "期末"]

# from_product() 生成所有组合
结果 = 每个学号 × 每个学期
    = (1001, 期中), (1001, 期末), 
      (1002, 期中), (1002, 期末), 
      ... 共10个组合
```

#### 数学公式
**总组合数 = len(stu_ids) × len(semisters) = 5 × 2 = 10**

### 📊 最终输出结构
```
              语文  数学  英语
学号   学期              
1001  期中     XX   XX   XX
      期末     XX   XX   XX  
1002  期中     XX   XX   XX
      期末     XX   XX   XX
1003  期中     XX   XX   XX
      期末     XX   XX   XX
1004  期中     XX   XX   XX
      期末     XX   XX   XX
1005  期中     XX   XX   XX
      期末     XX   XX   XX
```

### 💡 应用价值
- **数据组织**: 清晰表示学生在不同时期的成绩
- **分组分析**: 可按学号或学期进行分组统计
- **数据查询**: 支持多层级的数据检索

这种结构特别适合**教育数据分析**、**时间序列数据**等需要多维度索引的场景！


根据第一级索引分组数据，按照期中成绩占`25%`，期末成绩占`75%` 的方式计算每个学生每门课的成绩。

In [13]:
df.groupby(level=0).agg(lambda x: x.values[0] * 0.25 + x.values[1] * 0.75)

,语文,数学,英语
学号,,,
1001,84.25,66.75,87.75
1002,92.25,86.75,71.50
1003,93.25,100.00,69.50
1004,86.75,67.75,70.25
1005,71.75,84.25,77.50


## 这里的匿名函数中的x取到的是什么值？为什么可以x.values这样去取值？
在这个 `groupby().agg()` 操作中：

`x` 是什么：
- `x` 是每个分组（按学号分组）后的 `Series` 对象
- 对于每个学号，x 包含该学生的所有成绩数据（期中+期末）

为什么可以用 `x.values`：
- x 是 pandas Series，.values 属性返回底层的 NumPy 数组
- 由于每个学生只有2个成绩（期中、期末），所以 x.values 是长度为2的数组
- x.values[0] = 期中成绩，x.values[1] = 期末成绩

具体例子：

对于学号1001：
- x = Series([81, 76], index=['期中', '期末'])
- x.values = array([81, 76])
- 计算：81×0.25 + 76×0.75 = 77.25

这种写法利用了 pandas Series 的 .values 属性来快速访问数值，比用索引更简洁。

也就是说：`x['期中'] == x.values[0]` 和 `x['期末'] == x.values[1]`

所以上面的代码完全可以写成：

`df.groupby(level=0).agg(lambda x: x['期中'] * 0.25 + x['期末'] * 0.75)`

只不过，x.values[0]的写法更加简洁，x['期中']这样的写法更加语义化，更好理解。

### 间隔索引

间隔索引顾名思义是使用固定的间隔范围充当索引，我们通常会使用`interval_range`函数来创建间隔索引，代码如下所示。

In [14]:
index = pd.interval_range(0, 5)
index

IntervalIndex([(0, 1], (1, 2], (2, 3], (3, 4], (4, 5]], dtype='interval[int64, right]')

`IntervalIndex`有一个名为`contains`的方法，可以检查范围内是否包含了某个元素，如下所示。

In [16]:
index.contains(1.5)

array([False,  True, False, False, False])

`IntervalIndex`还有一个名为`overlaps`的方法，可以检查一个范围跟其他的范围是否有重叠，如下所示。

In [17]:
index.overlaps(pd.Interval(1.5, 3.5))

array([False,  True,  True,  True, False])

如果希望间隔范围是左闭右开的状态，可以在创建间隔索引时通过`closed='left'`来做到；如果希望两边都是关闭状态，可以将`close`参数的值赋值为`both`，代码如下所示。

In [22]:
index = pd.interval_range(0, 5, closed="left")
index

IntervalIndex([[0, 1), [1, 2), [2, 3), [3, 4), [4, 5)], dtype='interval[int64, left]')

In [23]:
index = pd.interval_range(
    start=pd.Timestamp("2022-01-01"), end=pd.Timestamp("2022-01-04"), closed="both"
)
index

IntervalIndex([[2022-01-01 00:00:00, 2022-01-02 00:00:00],
               [2022-01-02 00:00:00, 2022-01-03 00:00:00],
               [2022-01-03 00:00:00, 2022-01-04 00:00:00]],
              dtype='interval[datetime64[ns], both]')

### 日期时间索引

`DatetimeIndex`应该是众多索引中最复杂最重要的一种索引，我们通常会使用`date_range()`函数来创建日期时间索引，该函数有几个非常重要的参数`start`、`end`、`periods`、`freq`、`tz`，分别代表起始日期时间、结束日期时间、生成周期、采样频率和时区。我们先来看看如何创建`DatetimeIndex`对象，再来讨论它的相关运算和操作，代码如下所示。

In [24]:
pd.date_range("2021-1-1", "2021-6-30", periods=10) # periods：周期

DatetimeIndex(['2021-01-01', '2021-01-21', '2021-02-10', '2021-03-02',
               '2021-03-22', '2021-04-11', '2021-05-01', '2021-05-21',
               '2021-06-10', '2021-06-30'],
              dtype='datetime64[ns]', freq=None)

In [33]:
pd.date_range("2021-1-1", "2021-6-30", freq="H")

/var/folders/hd/pwqcx5pn30g7jq6160hcf9fr0000gp/T/ipykernel_1230/2200560922.py:1: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  pd.date_range("2021-1-1", "2021-6-30", freq="H")


DatetimeIndex(['2021-01-01 00:00:00', '2021-01-01 01:00:00',
               '2021-01-01 02:00:00', '2021-01-01 03:00:00',
               '2021-01-01 04:00:00', '2021-01-01 05:00:00',
               '2021-01-01 06:00:00', '2021-01-01 07:00:00',
               '2021-01-01 08:00:00', '2021-01-01 09:00:00',
               ...
               '2021-06-29 15:00:00', '2021-06-29 16:00:00',
               '2021-06-29 17:00:00', '2021-06-29 18:00:00',
               '2021-06-29 19:00:00', '2021-06-29 20:00:00',
               '2021-06-29 21:00:00', '2021-06-29 22:00:00',
               '2021-06-29 23:00:00', '2021-06-30 00:00:00'],
              dtype='datetime64[ns]', length=4321, freq='h')

>**说明**：`freq=W`表示采样周期为一周，它会默认星期日是一周的开始；如果你希望星期一表示一周的开始，你可以将其修改为`freq=W-MON`；你也可以试着将该参数的值修改为`12H`，`ME`，`Q`等，看看会发生什么，相信你不难猜到它们的含义。

## freq能填哪些值？
freq 参数可以填写以下常用值：

**时间频率：**
- 'D' - 天
- 'W' - 周（默认周日开始）
- 'W-MON' - 周（周一开始）
- 'ME' - 月末 month end
- 'MS' - 月初
- 'QE' - 季度末
- 'QS' - 季度初
- 'YE' - 年末
- 'YS' - 年初

**小时频率：**
- 'H' - 小时
- '12H' - 12小时
- 'T' 或 'min' - 分钟
- 'S' - 秒
- 'L' 或 'ms' - 毫秒
- 'U' 或 'us' - 微秒
- 'N' 或 'ns' - 纳秒

**偏移别名：**
- 'B' - 工作日
- 'BM' - 月末工作日
- 'BMS' - 月初工作日

**自定义频率：**
- '2D' - 每2天
- '3W' - 每3周
- '2H' - 每2小时
这些频率字符串可以组合使用，如 'W-MON' 表示以周一为开始的周频率。

`DatatimeIndex`可以跟`DateOffset`类型进行运算，这一点很好理解，以为我们可以设置一个时间差让时间向前或向后偏移，具体的操作如下所示。

In [38]:
index = pd.date_range("2021-1-1", "2021-6-30", freq="W-MON")
index
index - pd.DateOffset(days=2)

DatetimeIndex(['2021-01-02', '2021-01-09', '2021-01-16', '2021-01-23',
               '2021-01-30', '2021-02-06', '2021-02-13', '2021-02-20',
               '2021-02-27', '2021-03-06', '2021-03-13', '2021-03-20',
               '2021-03-27', '2021-04-03', '2021-04-10', '2021-04-17',
               '2021-04-24', '2021-05-01', '2021-05-08', '2021-05-15',
               '2021-05-22', '2021-05-29', '2021-06-05', '2021-06-12',
               '2021-06-19', '2021-06-26'],
              dtype='datetime64[ns]', freq=None)

## dateoffset有什么用？
`DateOffset` 是 pandas 中用于时间偏移操作的工具，主要用途包括：

**核心功能：**
- 对时间序列进行精确的时间偏移计算
- 支持各种时间单位（天、周、月、年等）
- 可以向前或向后偏移时间

**常见应用场景：**
1. 时间计算：如你的代码中 index - pd.DateOffset(days=2) 将所有日期向前偏移2天
2. 工作日计算：跳过周末和节假日
3. 月末/月初调整：将日期调整到月末或月初
4. 自定义时间间隔：创建不规则的时间序列

**优势：**
- 比简单的数值加减更智能（能处理月末、闰年等特殊情况）
- 支持复杂的时间偏移规则
- 与 DatetimeIndex 完美配合

在你的示例中，pd.DateOffset(days=2) 会将整个时间序列中的每个日期都减去2天，这在数据分析中常用于时间对齐或计算时间差。

In [39]:
index + pd.DateOffset(hours=2, minutes=10)

DatetimeIndex(['2021-01-04 02:10:00', '2021-01-11 02:10:00',
               '2021-01-18 02:10:00', '2021-01-25 02:10:00',
               '2021-02-01 02:10:00', '2021-02-08 02:10:00',
               '2021-02-15 02:10:00', '2021-02-22 02:10:00',
               '2021-03-01 02:10:00', '2021-03-08 02:10:00',
               '2021-03-15 02:10:00', '2021-03-22 02:10:00',
               '2021-03-29 02:10:00', '2021-04-05 02:10:00',
               '2021-04-12 02:10:00', '2021-04-19 02:10:00',
               '2021-04-26 02:10:00', '2021-05-03 02:10:00',
               '2021-05-10 02:10:00', '2021-05-17 02:10:00',
               '2021-05-24 02:10:00', '2021-05-31 02:10:00',
               '2021-06-07 02:10:00', '2021-06-14 02:10:00',
               '2021-06-21 02:10:00', '2021-06-28 02:10:00'],
              dtype='datetime64[ns]', freq=None)

如果`Series`对象或`DataFrame`对象使用了`DatetimeIndex`类型的索引，此时我们可以通过`asfreq()`方法指定一个时间频率来实现对数据的抽样，我们仍然以之前讲过的百度股票数据为例，给大家做一个演示。

In [46]:
baidu_df = pd.read_excel("./2022年股票数据.xlsx", sheet_name="BIDU", index_col="Date")
print(baidu_df)
baidu_df.sort_index(inplace=True)
baidu_df.asfreq("5D")

               Open      High     Low   Close   Volume
Date                                                  
2022-12-30  113.490  116.5000  113.15  114.38  1727642
2022-12-29  112.810  116.0600  111.30  115.10  1454617
2022-12-28  114.090  115.5300  109.88  111.60  1983757
2022-12-27  113.100  117.5000  112.48  116.48  2668445
2022-12-23  113.880  114.2500  111.52  111.61  1221825
...             ...       ...     ...     ...      ...
2022-01-07  152.980  157.0000  152.28  153.33  2751971
2022-01-06  146.195  153.0000  144.41  150.75  3839019
2022-01-05  143.820  150.2600  142.95  143.88  3505931
2022-01-04  148.140  148.4289  143.56  146.53  2876800
2022-01-03  148.910  149.9606  144.95  149.10  2330166

[251 rows x 5 columns]


,Open,High,Low,Close,Volume
Date,,,,,
2022-01-03,148.91,149.9606,144.950,149.10,2330166.0
2022-01-08,NaN,NaN,NaN,NaN,NaN
2022-01-13,155.62,157.6400,152.310,152.51,3271577.0
2022-01-18,150.94,157.4300,149.610,152.94,3187153.0
2022-01-23,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...
2022-12-09,123.26,124.1100,119.585,119.99,3470483.0
2022-12-14,119.46,120.3500,117.530,119.22,2527860.0
2022-12-19,114.14,114.6000,111.190,112.08,2059607.0


大家可能注意到了，每5天抽取1天有可能会抽中非交易日，那么对应的列都变成了空值，为了解决这个问题，在使用`asfreq`方法时可以通过`method`参数来指定一种填充空值的方法，可以将相邻的交易日的数据填入进来。

In [45]:
baidu_df.asfreq("5D", method="ffill")

,Open,High,Low,Close,Volume
Date,,,,,
2022-01-03,148.91,149.9606,144.950,149.10,2330166
2022-01-08,152.98,157.0000,152.280,153.33,2751971
2022-01-13,155.62,157.6400,152.310,152.51,3271577
2022-01-18,150.94,157.4300,149.610,152.94,3187153
2022-01-23,162.53,164.9600,156.260,156.84,5463935
...,...,...,...,...,...
2022-12-09,123.26,124.1100,119.585,119.99,3470483
2022-12-14,119.46,120.3500,117.530,119.22,2527860
2022-12-19,114.14,114.6000,111.190,112.08,2059607


当使用`DatetimeIndex`索引时，我们也可以通过`resample()`方法基于时间对数据进行重采样，相当于根据时间周期对数据进行了分组操作，分组之后还可以进行聚合统计，代码如下所示。

In [50]:
baidu_df.resample("ME").mean()

,Open,High,Low,Close,Volume
Date,,,,,
2022-01-31,151.834250,155.501470,148.755630,152.183000,3.498542e+06
2022-02-28,157.680263,161.643947,155.390863,158.938947,2.688915e+06
2022-03-31,143.395217,148.291517,138.510143,142.973043,6.411250e+06
2022-04-30,130.035250,132.492250,126.301830,128.803000,3.579267e+06
2022-05-31,121.388571,124.888419,118.335552,121.821429,3.322147e+06
2022-06-30,145.988095,148.762329,143.066910,145.682857,3.442716e+06
2022-07-31,143.916500,146.410655,140.965030,144.106000,2.078316e+06
2022-08-31,137.376087,140.525000,134.869565,137.872174,2.556926e+06
2022-09-30,127.932857,129.994524,126.203410,127.929048,2.257403e+06


In [55]:
baidu_df.resample("ME").agg(["mean", "min", "max"])

Open                         High                    \
                  mean      min     max        mean      min      max   
Date                                                                    
2022-01-31  151.834250  143.720  162.92  155.501470  146.670  165.020   
2022-02-28  157.680263  140.010  165.75  161.643947  151.350  171.870   
2022-03-31  143.395217  105.010  161.32  148.291517  113.500  170.350   
2022-04-30  130.035250  110.720  150.61  132.492250  114.980  154.295   
2022-05-31  121.388571  105.020  145.87  124.888419  110.955  146.990   
2022-06-30  145.988095  136.680  155.19  148.762329  139.450  156.770   
2022-07-31  143.916500  133.075  151.81  146.410655  137.380  156.690   
2022-08-31  137.376087  129.770  154.27  140.525000  130.720  155.480   
2022-09-30  127.932857  115.210  144.15  129.994524  119.120  145.910   
2022-10-31  101.171529   77.500  124.22  103.180000   78.420  125.230   
2022-11-30   90.401905   76.180  104.99   92.960662   80.090  109.880   
2022-12-31  114.969524  107.300  123.26  117.136900  110.000  124.110   

                   Low                          Close                  \
                  mean       min      max        mean     min     max   
Date                                                                    
2022-01-31  148.755630  139.1000  160.780  152.183000  143.73  162.03   
2022-02-28  155.390863  137.0700  165.370  158.938947  149.82  167.35   
2022-03-31  138.510143  102.1800  160.000  142.973043  108.38  162.86   
2022-04-30  126.301830  110.3000  145.600  128.803000  111.93  153.80   
2022-05-31  118.335552  101.6166  140.070  121.821429  106.09  140.35   
2022-06-30  143.066910  130.5100  151.820  145.682857  131.88  155.24   
2022-07-31  140.965030  131.0500  150.960  144.106000  136.57  154.69   
2022-08-31  134.869565  127.0500  147.470  137.872174  128.33  151.02   
2022-09-30  126.203410  115.1608  141.580  127.929048  117.49  144.48   
2022-10-31   98.575729   73.5801  122.330  100.523810   76.57  123.30   
2022-11-30   88.816119   75.7500  104.630   90.919048   78.13  108.60   
2022-12-31  112.689043  106.5800  119.585  114.680000  108.11  120.33   

                  Volume                     
                    mean      min       max  
Date                                         
2022-01-31  3.498542e+06  1907890   6628154  
2022-02-28  2.688915e+06  1864828   4260277  
2022-03-31  6.411250e+06  2942768  17773714  
2022-04-30  3.579267e+06  2399134   6534788  
2022-05-31  3.322147e+06  1648518   7224841  
2022-06-30  3.442716e+06  1944555   5293688  
2022-07-31  2.078316e+06  1224029   3206156  
2022-08-31  2.556926e+06  1144496   9243776  
2022-09-30  2.257403e+06  1676840   4042207  
2022-10-31  3.975162e+06   874741  16008458  
2022-11-30  3.601258e+06  1991505   7073554  
2022-12-31  3.406129e+06  1221825  13024757

**提示**：不知大家是否注意到，上面输出的`DataFrame` 的列索引是一个`MultiIndex`对象。你可以访问上面的`DataFrame`对象的`columns`属性看看。

In [56]:
baidu_df.resample("ME").agg(["mean", "min", "max"]).columns

MultiIndex([(  'Open', 'mean'),
            (  'Open',  'min'),
            (  'Open',  'max'),
            (  'High', 'mean'),
            (  'High',  'min'),
            (  'High',  'max'),
            (   'Low', 'mean'),
            (   'Low',  'min'),
            (   'Low',  'max'),
            ( 'Close', 'mean'),
            ( 'Close',  'min'),
            ( 'Close',  'max'),
            ('Volume', 'mean'),
            ('Volume',  'min'),
            ('Volume',  'max')],
           )

In [61]:
# baidu_df = baidu_df.tz_localize("Asia/Chongqing") # tz_localize() 用于添加时区信息，tz_convert() 用于转换时区。tz_localize() 方法只能对没有时区信息的 DatetimeIndex 使用一次。一旦 DatetimeIndex 已经有时区信息，再次调用 tz_localize() 就会报错。
baidu_df

,Open,High,Low,Close,Volume
Date,,,,,
2022-01-03 00:00:00+08:00,148.910,149.9606,144.95,149.10,2330166
2022-01-04 00:00:00+08:00,148.140,148.4289,143.56,146.53,2876800
2022-01-05 00:00:00+08:00,143.820,150.2600,142.95,143.88,3505931
2022-01-06 00:00:00+08:00,146.195,153.0000,144.41,150.75,3839019
2022-01-07 00:00:00+08:00,152.980,157.0000,152.28,153.33,2751971
...,...,...,...,...,...
2022-12-23 00:00:00+08:00,113.880,114.2500,111.52,111.61,1221825
2022-12-27 00:00:00+08:00,113.100,117.5000,112.48,116.48,2668445
2022-12-28 00:00:00+08:00,114.090,115.5300,109.88,111.60,1983757


在对时间本地化以后，我们再使用`tz_convert()`方法就可以实现转换时区，代码如下所示。

In [62]:
baidu_df.tz_convert("America/New_York")

,Open,High,Low,Close,Volume
Date,,,,,
2022-01-02 11:00:00-05:00,148.910,149.9606,144.95,149.10,2330166
2022-01-03 11:00:00-05:00,148.140,148.4289,143.56,146.53,2876800
2022-01-04 11:00:00-05:00,143.820,150.2600,142.95,143.88,3505931
2022-01-05 11:00:00-05:00,146.195,153.0000,144.41,150.75,3839019
2022-01-06 11:00:00-05:00,152.980,157.0000,152.28,153.33,2751971
...,...,...,...,...,...
2022-12-22 11:00:00-05:00,113.880,114.2500,111.52,111.61,1221825
2022-12-26 11:00:00-05:00,113.100,117.5000,112.48,116.48,2668445
2022-12-27 11:00:00-05:00,114.090,115.5300,109.88,111.60,1983757
